In [ ]:
# Colab: installer tesseract + langue française (si besoin)
!apt-get update -y
!apt-get install -y tesseract-ocr tesseract-ocr-fra


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,006 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy

In [ ]:
!pip install pytesseract


In [ ]:
!unzip generated_reports.zip -d /content/


Archive:  generated_reports.zip
   creating: /content/generated_reports/
  inflating: /content/generated_reports/rapport_1.png  
  inflating: /content/generated_reports/rapport_10.png  
  inflating: /content/generated_reports/rapport_100.png  
  inflating: /content/generated_reports/rapport_11.png  
  inflating: /content/generated_reports/rapport_12.png  
  inflating: /content/generated_reports/rapport_13.png  
  inflating: /content/generated_reports/rapport_14.png  
  inflating: /content/generated_reports/rapport_15.png  
  inflating: /content/generated_reports/rapport_16.png  
  inflating: /content/generated_reports/rapport_17.png  
  inflating: /content/generated_reports/rapport_18.png  
  inflating: /content/generated_reports/rapport_19.png  
  inflating: /content/generated_reports/rapport_2.png  
  inflating: /content/generated_reports/rapport_20.png  
  inflating: /content/generated_reports/rapport_21.png  
  inflating: /content/generated_reports/rapport_22.png  
  inflating: /co

In [ ]:
import json
from datasets import Dataset

json_file = "resultats_layoutlmv3_with_image_path.json"

with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# --- garder uniquement 80 images uniques ---
images_seen = set()
filtered_data = []

for item in data:
    image_name = item["image_path"]
    if image_name not in images_seen and len(images_seen) < 80:
        images_seen.add(image_name)
    # On garde quand même toutes les questions de l'image si on a décidé de la prendre
    if image_name in images_seen:
        filtered_data.append(item)

# --- créer le dataset HuggingFace ---
examples = []
for item in filtered_data:
    examples.append({
        "tokens": item["tokens"],
        "bboxes": item["bboxes"],
        "answers": item["answers"],
        "start_positions": item["start_positions"],
        "end_positions": item["end_positions"],
        "image_path": item["image_path"]
    })

dataset = Dataset.from_list(examples)

print(f"Nombre d'images uniques utilisées : {len(images_seen)}")
print(dataset[0])


Nombre d'images uniques utilisées : 80
{'tokens': ['LEUCOCYTES', '7600', '/mm°', '(4000', '-', '10000)'], 'bboxes': [[99, 750, 249, 783], [553, 755, 599, 773], [605, 755, 646, 769], [981, 755, 1026, 772], [1031, 763, 1037, 765], [1046, 755, 1097, 772]], 'answers': ['7600'], 'start_positions': [1], 'end_positions': [1], 'image_path': 'generated_reports/rapport_1.png'}


In [ ]:
import json
from datasets import Dataset

json_file = "resultats_layoutlmv3_with_image_path.json"

with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Préparer le dataset
dataset_dict = {
    "input_ids": [],   # placeholder pour tokenizer
    "attention_mask": [], # placeholder pour tokenizer
    "token_type_ids": [], # placeholder si nécessaire
    "bbox": [],        # positions des tokens
    "labels": []       # positions de réponse
}

# Pour simplifier, on va créer un dataset minimal avec tokens, bboxes et positions
examples = []
for item in data:
    examples.append({
        "tokens": item["tokens"],
        "bboxes": item["bboxes"],
        "answers": item["answers"],
        "start_positions": item["start_positions"],
        "end_positions": item["end_positions"],
        "image_path": item["image_path"]
    })

dataset = Dataset.from_list(examples)
print(dataset[0])


{'tokens': ['LEUCOCYTES', '7600', '/mm°', '(4000', '-', '10000)'], 'bboxes': [[99, 750, 249, 783], [553, 755, 599, 773], [605, 755, 646, 769], [981, 755, 1026, 772], [1031, 763, 1037, 765], [1046, 755, 1097, 772]], 'answers': ['7600'], 'start_positions': [1], 'end_positions': [1], 'image_path': 'generated_reports/rapport_1.png'}


In [ ]:
from transformers import LayoutLMv3Processor, LayoutLMv3ForQuestionAnswering
import torch

model_name = "microsoft/layoutlmv3-base"
processor = LayoutLMv3Processor.from_pretrained(
    model_name,
    apply_ocr=False  # IMPORTANT : on fournit nos propres bboxes
)
model = LayoutLMv3ForQuestionAnswering.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of LayoutLMv3ForQuestionAnswering were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['qa_outputs.dense.bias', 'qa_outputs.dense.weight', 'qa_outputs.out_proj.bias', 'qa_outputs.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def normalize_bbox(box, width, height):
    """
    Convertit une bbox en pixels vers le format attendu (0–1000).
    box = [x0, y0, x1, y1] en pixels
    """
    return [
        int(1000 * (box[0] / width)),
        int(1000 * (box[1] / height)),
        int(1000 * (box[2] / width)),
        int(1000 * (box[3] / height)),
    ]


In [ ]:
from transformers import LayoutLMv3Processor, LayoutLMv3ForQuestionAnswering
from PIL import Image
import torch

def preprocess_function(example):
    # Charger l'image
    image = Image.open(example["image_path"]).convert("RGB")
    width, height = image.size

    # Normaliser les bboxes
    bboxes = [normalize_bbox(box, width, height) for box in example["bboxes"]]

    # Récup tokens
    words = [str(w) for w in example["tokens"]]

    # Encodage
    encoded_inputs = processor(
        images=image,
        text=words,
        boxes=bboxes,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    # Labels
    encoded_inputs["start_positions"] = torch.tensor(example["start_positions"][0])
    encoded_inputs["end_positions"] = torch.tensor(example["end_positions"][0])

    # Retirer batch dim
    for k in encoded_inputs:
        encoded_inputs[k] = encoded_inputs[k].squeeze(0)

    return encoded_inputs


# ⚠ batched=False pour traiter un exemple à la fois
processed_dataset = dataset.map(
    preprocess_function,
    batched=False,
    remove_columns=dataset.column_names   # 🔑 supprime le texte brut
)

# Vérification
print(processed_dataset[0])


Map:   0%|          | 0/1300 [00:00<?, ? examples/s]

{'start_positions': 1, 'end_positions': 1, 'input_ids': [0, 10611, 12945, 4571, 975, 565, 1723, 262, 4697, 1589, 5471, 12938, 36, 38161, 111, 45563, 43, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./layoutlmv3_model",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    learning_rate=5e-5,
    logging_steps=100,
    remove_unused_columns=False,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    tokenizer=processor,
)

trainer.train()


/tmp/ipython-input-1013323965.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eya-fakhfakhh (eya-fakhfakhh-enetcom) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Step,Training Loss
100,1.035100
200,0.036000
300,0.000200
400,0.000100
500,0.000100
600,0.000100
700,0.000000
800,0.000000
900,0.000000
1000,0.000000


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


TrainOutput(global_step=1040, training_loss=0.1030402016550607, metrics={'train_runtime': 27320.0874, 'train_samples_per_second': 0.076, 'train_steps_per_second': 0.038, 'total_flos': 552048286433280.0, 'train_loss': 0.1030402016550607, 'epoch': 2.0})

In [ ]:
OUTPUT_DIR = "./layoutlmv3_finetuned"

# Après avoir fini l'entraînement
model.save_pretrained(OUTPUT_DIR)
processor.save_pretrained(OUTPUT_DIR)


[]

In [ ]:
import shutil
from google.colab import files

# Compresser le dossier en ZIP
shutil.make_archive("layoutlmv3_finetuned", 'zip', "layoutlmv3_finetuned")


'/content/layoutlmv3_finetuned.zip'

In [ ]:

# Télécharger le ZIP sur ton PC
files.download("layoutlmv3_finetuned.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformers import LayoutLMv3Processor
from PIL import Image
import torch

# Charger le modèle fine-tuné
model = LayoutLMv3ForQuestionAnswering.from_pretrained("./layoutlmv3_finetuned")
model.eval()

# Exemple de test (prends un exemple de ton dataset)
test_example = dataset[0]
image = Image.open(test_example["image_path"]).convert("RGB")
width, height = image.size

# Normaliser les bboxes
bboxes = [normalize_bbox(box, width, height) for box in test_example["bboxes"]]
words = [str(w) for w in test_example["tokens"]]

# Préparer les inputs
inputs = processor(
    images=image,
    text=words,
    boxes=bboxes,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

# Inférence
with torch.no_grad():
    outputs = model(**inputs)

start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Trouver les positions prédictes
start_idx = torch.argmax(start_logits, dim=1).item()
end_idx = torch.argmax(end_logits, dim=1).item()

# Extraire la réponse à partir des tokens
predicted_tokens = words[start_idx : end_idx + 1]
predicted_answer = " ".join(predicted_tokens)

print("✅ Réponse prédite :", predicted_answer)
print("🎯 Réponse attendue :", " ".join(test_example["answers"]))


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


✅ Réponse prédite : 7600
🎯 Réponse attendue : 7600


In [ ]:
from transformers import LayoutLMv3Processor
from PIL import Image
import torch

# Charger le modèle fine-tuné
model = LayoutLMv3ForQuestionAnswering.from_pretrained("./layoutlmv3_finetuned")
model.eval()

# Exemple de test (prends un exemple de ton dataset)
test_example = dataset[1]
image = Image.open(test_example["image_path"]).convert("RGB")
width, height = image.size

# Normaliser les bboxes
bboxes = [normalize_bbox(box, width, height) for box in test_example["bboxes"]]
words = [str(w) for w in test_example["tokens"]]

# Préparer les inputs
inputs = processor(
    images=image,
    text=words,
    boxes=bboxes,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

# Inférence
with torch.no_grad():
    outputs = model(**inputs)

start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Trouver les positions prédictes
start_idx = torch.argmax(start_logits, dim=1).item()
end_idx = torch.argmax(end_logits, dim=1).item()

# Extraire la réponse à partir des tokens
predicted_tokens = words[start_idx : end_idx + 1]
predicted_answer = " ".join(predicted_tokens)

print("✅ Réponse prédite :", predicted_answer)
print("🎯 Réponse attendue :", " ".join(test_example["answers"]))


✅ Réponse prédite : 21400
🎯 Réponse attendue : 21400


In [ ]:
from transformers import LayoutLMv3Processor
from PIL import Image
import torch
processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)
# Charger le modèle fine-tuné
model = LayoutLMv3ForQuestionAnswering.from_pretrained("./layoutlmv3_finetuned")
model.eval()

# Exemple de test (prends un exemple de ton dataset)
test_example = dataset[4]
image = Image.open(test_example["image_path"]).convert("RGB")
width, height = image.size

# Normaliser les bboxes
bboxes = [normalize_bbox(box, width, height) for box in test_example["bboxes"]]
words = [str(w) for w in test_example["tokens"]]

# Préparer les inputs
inputs = processor(
    images=image,
    text=words,
    boxes=bboxes,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

# Inférence
with torch.no_grad():
    outputs = model(**inputs)

start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Trouver les positions prédictes
start_idx = torch.argmax(start_logits, dim=1).item()
end_idx = torch.argmax(end_logits, dim=1).item()

# Extraire la réponse à partir des tokens
predicted_tokens = words[start_idx : end_idx + 1]
predicted_answer = " ".join(predicted_tokens)

print("✅ Réponse prédite :", predicted_answer)
print("🎯 Réponse attendue :", " ".join(test_example["answers"]))


✅ Réponse prédite : 66.8%
🎯 Réponse attendue : 66.8


In [ ]:
from transformers import LayoutLMv3Processor, LayoutLMv3ForQuestionAnswering
from PIL import Image
import torch

processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)
model = LayoutLMv3ForQuestionAnswering.from_pretrained("./layoutlmv3_finetuned")
model.eval()

def normalize_bbox(box, width, height):
    # Si tu as déjà une fonction de normalisation, sinon :
    return [
        int(1000 * (box[0] / width)),
        int(1000 * (box[1] / height)),
        int(1000 * (box[2] / width)),
        int(1000 * (box[3] / height))
    ]

# Tester les exemples de 1 à 13
for i in range(1, 14):
    test_example = dataset[i]
    image = Image.open(test_example["image_path"]).convert("RGB")
    width, height = image.size

    # Normaliser les bboxes
    bboxes = [normalize_bbox(box, width, height) for box in test_example["bboxes"]]
    words = [str(w) for w in test_example["tokens"]]

    # Préparer les inputs
    inputs = processor(
        images=image,
        text=words,
        boxes=bboxes,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    # Inférence
    with torch.no_grad():
        outputs = model(**inputs)

    start_idx = torch.argmax(outputs.start_logits, dim=1).item()
    end_idx = torch.argmax(outputs.end_logits, dim=1).item()

    predicted_tokens = words[start_idx : end_idx + 1]
    predicted_answer = " ".join(predicted_tokens)

    print(f"Exemple {i}:")
    print("✅ Réponse prédite :", predicted_answer)
    print("🎯 Réponse attendue :", " ".join(test_example["answers"]))
    print("-" * 40)


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Exemple 1:
✅ Réponse prédite : 21400
🎯 Réponse attendue : 21400
----------------------------------------
Exemple 2:
✅ Réponse prédite : 14.1
🎯 Réponse attendue : 14.1
----------------------------------------
Exemple 3:
✅ Réponse prédite : 41.8
🎯 Réponse attendue : 41.8
----------------------------------------
Exemple 4:
✅ Réponse prédite : 66.8%
🎯 Réponse attendue : 66.8
----------------------------------------
Exemple 5:
✅ Réponse prédite : 21.1%
🎯 Réponse attendue : 21.1
----------------------------------------
Exemple 6:
✅ Réponse prédite : 4.4
🎯 Réponse attendue : 4.4
----------------------------------------
Exemple 7:
✅ Réponse prédite : 0.5
🎯 Réponse attendue : 0.5
----------------------------------------
Exemple 8:
✅ Réponse prédite : 7.1
🎯 Réponse attendue : 7.1
----------------------------------------
Exemple 9:
✅ Réponse prédite : 4.73
🎯 Réponse attendue : 4.73
----------------------------------------
Exemple 10:
✅ Réponse prédite : 88.5
🎯 Réponse attendue : 88.5
------------

In [ ]:
import re
import unicodedata
import cv2
import pytesseract
from difflib import get_close_matches
import os
import json

# ---------------- CONFIG ----------------
pytesseract.pytesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
image_dir = r"."  # dossier contenant les images
output_json = "resultats_layoutlmv3_corrige1.json"

unites_valides = {
    "g/dL", "pg", "µm³", "um³", "fL", "%", "10^3/uL", "10^6/uL",
    "mm³", "10*3/uL", "10*6/uL", "/mm³", "10*3/mm³", "10*6/mm³", "/ml", "g/dl", "Pg",
    "10*6/µl", "/µl", "/ul", "millions/mm", "M/mm³", "femtolites", "10⁶/mm³",
    "10p6/mm³", "10p3/mm³", "10 p 6/mm³", "10 p 3/mm³"
}

champs_cibles = [
    "leucocytes", "hématies", "plaquettes", "hémoglobine", "hématocrite",
    "neutrophiles", "lymphocytes", "eosinophiles", "basophiles", "monocytes",
    "globules rouges", "globules blancs", "vgm", "ccmh", "tcmh"
]

champs_sans_accents = [
    re.sub(r'[^a-z0-9\s]', '', unicodedata.normalize('NFD', c).lower())
    for c in champs_cibles
]

questions = {c: f"Quelle est la valeur des {c} ?" if c not in {"vgm","ccmh","tcmh"}
             else f"Quelle est la valeur du {c} ?" for c in champs_cibles}

# ---------------- FONCTIONS UTILES ----------------
def normaliser(mot):
    mot = unicodedata.normalize('NFD', mot)
    return ''.join(c for c in mot if unicodedata.category(c) != 'Mn').lower()

def nettoyer_et_enlever_accents(texte):
    texte = unicodedata.normalize('NFD', texte.lower())
    return re.sub(r'[^a-z0-9\s]', '', ''.join(c for c in texte if unicodedata.category(c) != 'Mn'))

def est_unite_complexe(mot):
    pattern = r'^\d+p\d+\/mm[3³]$'
    pattern_general = r'^\d+[a-z]+\d+\/[a-z0-9/³*^]+$'
    return bool(re.match(pattern, mot.lower()) or re.match(pattern_general, mot.lower()))

def separer_chiffres_unites(texte):
    mots = texte.split()
    mots_corriges = []
    for mot in mots:
        if est_unite_complexe(mot):
            mots_corriges.append(mot)
        else:
            mot_sep = re.sub(r'(\d)([a-zA-Zµ/%])', r'\1 \2', mot)
            mot_sep = re.sub(r'([a-zA-Zµ/%])(\d)', r'\1 \2', mot_sep)
            mots_corriges.append(mot_sep)
    return ' '.join(mots_corriges)

def corriger_mot_ocr(mot):
    mot = mot.replace("'", "").replace("`", "")
    mot_clean = nettoyer_et_enlever_accents(mot)
    match = get_close_matches(mot_clean, champs_sans_accents, n=1, cutoff=0.7)
    return champs_cibles[champs_sans_accents.index(match[0])] if match else mot

def corriger_unite_intelligente(mot):
    mot_normalise = normaliser(mot)
    correspondances = get_close_matches(mot_normalise, [normaliser(u) for u in unites_valides], n=1, cutoff=0.6)
    if correspondances:
        index = [normaliser(u) for u in unites_valides].index(correspondances[0])
        return list(unites_valides)[index]
    return mot

def corriger_ligne_intelligente(ligne):
    ligne = separer_chiffres_unites(ligne)
    mots = ligne.split()
    mots_corriges = []
    for mot in mots:
        mot_corrige = corriger_mot_ocr(mot)
        if mot_corrige == mot:
            mot_corrige = corriger_unite_intelligente(mot)
        mots_corriges.append(mot_corrige)
    return ' '.join(mots_corriges)

# ---------------- CORRECTION EXTRACTION VALEUR ----------------
def extraire_valeurs_selon_champ(champ, ligne):
    ligne_sep = separer_chiffres_unites(ligne)
    valeurs = re.findall(r'\d+[.,]?\d*', ligne_sep)
    valeurs = [v.replace(',', '.') for v in valeurs]
    if not valeurs:
        return None
    return valeurs[0]  # ✅ On prend uniquement la première valeur

# Fusionner les nombres séparés en un seul token
def fusionner_chiffres(tokens):
    nouvelle_liste = []
    i = 0
    while i < len(tokens):
        if i+1 < len(tokens) and tokens[i].replace(',', '').isdigit() and tokens[i+1].replace(',', '').isdigit():
            fusion = tokens[i] + tokens[i+1]
            nouvelle_liste.append(fusion)
            i += 2
        else:
            nouvelle_liste.append(tokens[i])
            i += 1
    return nouvelle_liste

# ---------------- TRAITEMENT DES IMAGES ----------------
resultats = []

for filename in os.listdir(image_dir):
    if not filename.lower().endswith((".png", ".jpg", ".jpeg")):
        continue
    print(f"Traitement de {filename}...")
    image_path = os.path.join(image_dir, filename)
    image = cv2.imread(image_path)
    if image is None:
        print(f"⚠️ Impossible de lire {filename}, passage au suivant.")
        continue

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    data = pytesseract.image_to_data(gray, output_type=pytesseract.Output.DICT, lang='fra')
    n_boxes = len(data['text'])

    # Regroupement mots par ligne
    lines = {}
    tolerance_y = 15
    for i in range(n_boxes):
        mot = data['text'][i].strip()
        conf = int(data['conf'][i])
        if mot == '' or conf < 0:
            continue
        y, x = data['top'][i], data['left'][i]
        found = False
        for key in lines:
            if abs(key - y) <= tolerance_y:
                lines[key].append({
                    'word': mot,
                    'x': x,
                    'left': data['left'][i],
                    'top': data['top'][i],
                    'width': data['width'][i],
                    'height': data['height'][i]
                })
                found = True
                break
        if not found:
            lines[y] = [{
                'word': mot,
                'x': x,
                'left': data['left'][i],
                'top': data['top'][i],
                'width': data['width'][i],
                'height': data['height'][i]
            }]

    # Lignes corrigées
    lignes_corrigees = []
    lignes_mots_pos = []
    for y in sorted(lines):
        mots_tries = sorted(lines[y], key=lambda m: m['x'])
        ligne = " ".join(m['word'] for m in mots_tries)
        ligne_corrigee = corriger_ligne_intelligente(ligne)
        lignes_corrigees.append(ligne_corrigee)
        lignes_mots_pos.append(mots_tries)

    # Construction des QAs
    qas = []
    for champ, question in questions.items():
        for idx, ligne in enumerate(lignes_corrigees):
            if nettoyer_et_enlever_accents(champ) in nettoyer_et_enlever_accents(ligne):
                valeur = extraire_valeurs_selon_champ(champ, ligne)
                if valeur:
                    mots_pos = lignes_mots_pos[idx]
                    tokens = [m['word'] for m in mots_pos]
                    tokens = fusionner_chiffres(tokens)
                    bboxes = [[m['left'], m['top'], m['left']+m['width'], m['top']+m['height']] for m in mots_pos]

                    # Trouver l’index du token réponse
                    start_idx = None
                    for i, tok in enumerate(tokens):
                        tok_norm = tok.replace(",", ".").replace(":", "").replace(" ", "").lower()
                        val_norm = valeur.replace(",", ".").replace(" ", "").lower()
                        if val_norm in tok_norm:
                            start_idx = i
                            break

                    if start_idx is None:
                        continue

                    qas.append({
                        "id": f"{filename}_{champ}",
                        "question": question,
                        "tokens": tokens,
                        "bboxes": bboxes,
                        "answers": [valeur],
                        "start_positions": [start_idx],
                        "end_positions": [start_idx]
                    })
                break

    resultats.extend(qas)

# ---------------- SAUVEGARDE ----------------
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(resultats, f, ensure_ascii=False, indent=2)

print(f"✅ Extraction terminée. Résultats sauvegardés dans {output_json}")


Traitement de 1.jpg...
✅ Extraction terminée. Résultats sauvegardés dans resultats_layoutlmv3_corrige1.json


In [ ]:
import json

input_json = "resultats_layoutlmv3_corrige1.json"
output_json = "resultats_layoutlmv3_corrige_final_corrected.json"

with open(input_json, "r", encoding="utf-8") as f:
    data = json.load(f)

# Champs à corriger uniquement
champs_cibles = {"leucocytes", "plaquettes"}

for qa in data:
    # Vérifier si le champ est concerné
    for champ in champs_cibles:
        if champ in qa["question"].lower():
            tokens = qa["tokens"]
            start_idx = qa["start_positions"][0]
            end_idx = qa["end_positions"][0]

            # Extraire la valeur à partir des tokens
            token_valeur = "".join(tokens[start_idx:end_idx + 1]).replace(" ", "")

            # Mettre à jour la réponse
            qa["answers"] = [token_valeur]
            break  # On ne vérifie pas les autres champs

# Sauvegarder le résultat corrigé
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"✅ Les réponses pour leucocytes et plaquettes ont été corrigées et sauvegardées dans {output_json}")


✅ Les réponses pour leucocytes et plaquettes ont été corrigées et sauvegardées dans resultats_layoutlmv3_corrige_final_corrected.json


In [ ]:
import json
from transformers import LayoutLMv3Processor, LayoutLMv3ForQuestionAnswering
from PIL import Image
import torch

# Charger le processor et le modèle fine-tuné
processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)
model = LayoutLMv3ForQuestionAnswering.from_pretrained("./layoutlmv3_finetuned")
model.eval()

# Charger les exemples
with open("resultats_layoutlmv3_corrige_final_corrected.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Normaliser bboxes
def normalize_bbox(bbox, width, height):
    return [
        int(1000 * (bbox[0] / width)),
        int(1000 * (bbox[1] / height)),
        int(1000 * (bbox[2] / width)),
        int(1000 * (bbox[3] / height))
    ]

# Fusionner tokens numériques consécutifs et leurs bboxes
def merge_tokens(tokens, bboxes):
    merged_tokens = []
    merged_bboxes = []
    skip_next = False

    for i in range(len(tokens)):
        if skip_next:
            skip_next = False
            continue

        tok = tokens[i]
        bbox = bboxes[i]

        # Fusion des chiffres consécutifs
        if tok.isdigit() and i+1 < len(tokens) and tokens[i+1].isdigit():
            new_tok = tok + tokens[i+1]
            new_bbox = [
                min(bbox[0], bboxes[i+1][0]),
                min(bbox[1], bboxes[i+1][1]),
                max(bbox[2], bboxes[i+1][2]),
                max(bbox[3], bboxes[i+1][3])
            ]
            merged_tokens.append(new_tok)
            merged_bboxes.append(new_bbox)
            skip_next = True
        else:
            merged_tokens.append(tok)
            merged_bboxes.append(bbox)

    return merged_tokens, merged_bboxes

# Boucle sur tous les exemples
for example in data:
    image_path = example["id"].split("_")[0]  # Assure-toi que le chemin est correct
    image = Image.open(image_path).convert("RGB")
    width, height = image.size

    # Normaliser bboxes
    bboxes = [normalize_bbox(box, width, height) for box in example["bboxes"]]
    tokens = example["tokens"]
    question = example["question"]

    # Fusionner tokens numériques
    tokens, bboxes = merge_tokens(tokens, bboxes)

    # ⚠️ Vérifier que tokens et bboxes ont même longueur
    if len(tokens) != len(bboxes):
        print(f"⚠️ Skipping {example['id']} car tokens ({len(tokens)}) != bboxes ({len(bboxes)})")
        continue

    # Préparer les inputs
    inputs = processor(
        image,
        text=[tokens],      # déjà tokenisé → doit être List[str]
        boxes=[bboxes],     # mêmes dimensions que tokens
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    # Inférence
    with torch.no_grad():
        outputs = model(**inputs)

    start_idx = torch.argmax(outputs.start_logits, dim=1).item()
    end_idx = torch.argmax(outputs.end_logits, dim=1).item()

    predicted_tokens = tokens[start_idx:end_idx+1]
    predicted_answer = " ".join(predicted_tokens)

    print("=========================================")
    print("✅ Question :", question)
    print("📝 Réponse prédite :", predicted_answer)
    print("🎯 Réponse attendue :", " ".join(example["answers"]))


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


✅ Question : Quelle est la valeur des leucocytes ?
📝 Réponse prédite : 4000
🎯 Réponse attendue : 4000
✅ Question : Quelle est la valeur des plaquettes ?
📝 Réponse prédite : 192000
🎯 Réponse attendue : 192000
✅ Question : Quelle est la valeur des hémoglobine ?
📝 Réponse prédite : 13.7
🎯 Réponse attendue : 13.7
✅ Question : Quelle est la valeur des hématocrite ?
📝 Réponse prédite : (36-46)
🎯 Réponse attendue : 42.1
✅ Question : Quelle est la valeur des neutrophiles ?
📝 Réponse prédite : 65%
🎯 Réponse attendue : 65
✅ Question : Quelle est la valeur des lymphocytes ?
📝 Réponse prédite : 26%
🎯 Réponse attendue : 26
✅ Question : Quelle est la valeur des eosinophiles ?
📝 Réponse prédite : 1%
🎯 Réponse attendue : 1
✅ Question : Quelle est la valeur des basophiles ?
📝 Réponse prédite : 0%
🎯 Réponse attendue : 0
✅ Question : Quelle est la valeur des monocytes ?
📝 Réponse prédite : 8%
🎯 Réponse attendue : 8
✅ Question : Quelle est la valeur des globules rouges ?
📝 Réponse prédite : 4.63
🎯 Réponse

In [ ]:
!unzip generated_reports.zip -d /content/


In [ ]:
processed_dataset = dataset.map(
    preprocess_function,
    batched=False,
    remove_columns=[col for col in dataset.column_names if col != "image_path"]  # on garde image_path
)


Map:   0%|          | 0/1300 [00:00<?, ? examples/s]

In [ ]:
import zipfile
import os

# Chemin du fichier ZIP et dossier de destination
zip_path = "layoutlmv3_finetuned.zip"
extract_dir = "layoutlmv3_finetuned"

# Vérifier si le dossier existe déjà, sinon le créer
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Dézipper
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Modèle dézippé dans : {extract_dir}")


Modèle dézippé dans : layoutlmv3_finetuned


In [ ]:
from PIL import Image
import torch

def test_model(dataset, n=10):
    for i in range(min(n, len(dataset))):
        example = dataset[i]
        image = Image.open(example["image_path"]).convert("RGB")
        width, height = image.size

        # Normaliser les bboxes
        bboxes = [normalize_bbox(box, width, height) for box in example["bboxes"]]
        words = [str(w) for w in example["tokens"]]

        # Préparer les inputs pour le modèle
        inputs = processor(
            images=image,
            text=words,
            boxes=bboxes,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        with torch.no_grad():
            outputs = model(**inputs)

        start_idx = torch.argmax(outputs.start_logits, dim=1).item()
        end_idx = torch.argmax(outputs.end_logits, dim=1).item()

        predicted_tokens = words[start_idx:end_idx+1]
        predicted_answer = " ".join(predicted_tokens)

        print(f"Exemple {i+1}:")
        print("✅ Réponse prédite :", predicted_answer)
        print("🎯 Réponse attendue :", " ".join(example["answers"]))
        print("-"*40)

# Lancer le test
test_model(dataset, n=10)


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:1742: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Exemple 1:
✅ Réponse prédite : 7600
🎯 Réponse attendue : 7600
----------------------------------------
Exemple 2:
✅ Réponse prédite : 21400
🎯 Réponse attendue : 21400
----------------------------------------
Exemple 3:
✅ Réponse prédite : 14.1
🎯 Réponse attendue : 14.1
----------------------------------------
Exemple 4:
✅ Réponse prédite : 41.8
🎯 Réponse attendue : 41.8
----------------------------------------
Exemple 5:
✅ Réponse prédite : 66.8%
🎯 Réponse attendue : 66.8
----------------------------------------
Exemple 6:
✅ Réponse prédite : 21.1%
🎯 Réponse attendue : 21.1
----------------------------------------
Exemple 7:
✅ Réponse prédite : 4.4
🎯 Réponse attendue : 4.4
----------------------------------------
Exemple 8:
✅ Réponse prédite : 0.5
🎯 Réponse attendue : 0.5
----------------------------------------
Exemple 9:
✅ Réponse prédite : 7.1
🎯 Réponse attendue : 7.1
----------------------------------------
Exemple 10:
✅ Réponse prédite : 4.73
🎯 Réponse attendue : 4.73
------------

In [ ]:
from transformers import LayoutLMv3Processor, LayoutLMv3ForQuestionAnswering
from PIL import Image
import torch
import re

# Charger le modèle finetuné et le processor
OUTPUT_DIR = "./layoutlmv3_finetuned"
processor = LayoutLMv3Processor.from_pretrained(OUTPUT_DIR, apply_ocr=False)
model = LayoutLMv3ForQuestionAnswering.from_pretrained(OUTPUT_DIR)
model.eval()

# Normalisation des bboxes
def normalize_bbox(box, width, height):
    return [
        int(1000 * (box[0] / width)),
        int(1000 * (box[1] / height)),
        int(1000 * (box[2] / width)),
        int(1000 * (box[3] / height)),
    ]

# Nettoyage et normalisation des réponses pour EM/F1
def normalize_answer(s):
    # Supprime ponctuation et unités (% / mm³ / etc.)
    s = re.sub(r"[^0-9a-zA-Z.]+", " ", s)
    s = s.lower().strip()
    return s

# Calcul EM et F1
def compute_metrics(pred_answer, true_answer):
    pred_norm = normalize_answer(pred_answer)
    true_norm = normalize_answer(true_answer)

    # EM
    em = int(pred_norm == true_norm)

    # F1 token-level
    pred_tokens = pred_norm.split()
    true_tokens = true_norm.split()

    common = set(pred_tokens) & set(true_tokens)
    num_common = len(common)

    if num_common == 0:
        f1 = 0.0
    else:
        precision = num_common / len(pred_tokens)
        recall = num_common / len(true_tokens)
        f1 = 2 * (precision * recall) / (precision + recall)

    return em, f1

# Test sur le dataset
def test_model(dataset, n=None):
    total_em = 0
    total_f1 = 0
    n_examples = len(dataset) if n is None else min(n, len(dataset))

    for i in range(n_examples):
        example = dataset[i]
        image = Image.open(example["image_path"]).convert("RGB")
        width, height = image.size
        bboxes = [normalize_bbox(box, width, height) for box in example["bboxes"]]
        words = [str(w) for w in example["tokens"]]

        inputs = processor(
            images=image,
            text=words,
            boxes=bboxes,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        with torch.no_grad():
            outputs = model(**inputs)

        start_idx = torch.argmax(outputs.start_logits, dim=1).item()
        end_idx = torch.argmax(outputs.end_logits, dim=1).item()
        predicted_tokens = words[start_idx:end_idx+1]
        predicted_answer = " ".join(predicted_tokens)
        true_answer = " ".join(example["answers"])

        em, f1 = compute_metrics(predicted_answer, true_answer)
        total_em += em
        total_f1 += f1

        print(f"Exemple {i+1}:")
        print("✅ Prédit :", predicted_answer)
        print("🎯 Attendu :", true_answer)
        print(f"EM={em}, F1={f1:.2f}")
        print("-"*40)

    print(f"Exact Match moyen: {total_em/n_examples:.2f}")
    print(f"F1-score moyen: {total_f1/n_examples:.2f}")

# Lancer le test sur tout le dataset
test_model(processed_dataset)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
🎯 Attendu : 47100
EM=1, F1=1.00
----------------------------------------
Exemple 302:
✅ Prédit : 9.3
🎯 Attendu : 9.3
EM=1, F1=1.00
----------------------------------------
Exemple 303:
✅ Prédit : 29.4
🎯 Attendu : 29.4
EM=1, F1=1.00
----------------------------------------
Exemple 304:
✅ Prédit : 24.1%
🎯 Attendu : 24.1
EM=1, F1=1.00
----------------------------------------
Exemple 305:
✅ Prédit : 57.7%
🎯 Attendu : 57.7
EM=1, F1=1.00
----------------------------------------
Exemple 306:
✅ Prédit : 11.4%
🎯 Attendu : 11.4
EM=1, F1=1.00
----------------------------------------
Exemple 307:
✅ Prédit : 0.6
🎯 Attendu : 0.6
EM=1, F1=1.00
----------------------------------------
Exemple 308:
✅ Prédit : 6.2
🎯 Attendu : 6.2
EM=1, F1=1.00
----------------------------------------
Exemple 309:
✅ Prédit : 4.77
🎯 Attendu : 4.77
EM=1, F1=1.00
----------------------------------------
Exemple 310:
✅ Prédit : 61.5
🎯 Attendu : 61.